In [2]:
# https://blog.csdn.net/u013066244/article/details/132014791
from dotenv import load_dotenv
load_dotenv()

#>>> Uncomment the following line if you need to initialize FAISS with no AVX2 optimization
#>>> 如果您需要在没有 AVX2 优化的情况下初始化 FAISS，请取消以下注释
# os.environ['FAISS_NO_AVX2'] = '1'

# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../../text_files/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

#>>> embeddings = OpenAIEmbeddings()
# embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

db = FAISS.from_documents(docs, embedding)

query = "What did the president say about Ketanji Brown Jackson"

# >>> basic search
# docs = db.similarity_search(query)
# print(docs[0].page_content)

#>>> 有一些 FAISS 特定方法。其中之一是similarity_search_with_score，它不仅允许您返回文档，还允许返回查询到它们的距离分数。返回的距离分数是L2距离。因此，分数越低越好。

docs_and_scores = db.similarity_search_with_score(query, k=4) # default k=4
docs_and_scores[1][0].metadata.update({"test": "test"})
print(docs_and_scores[1][0])  # 1st element is the document, 2nd element is the score
print(len(docs_and_scores[1]))

#>>> 还可以使用similarity_search_by_vector 与给定嵌入向量相似的文档进行搜索，该向量接受嵌入向量作为参数而不是字符串
# embed 向量
# embedding_vector = embedding.embed_query(query)
# docs = db.similarity_search_by_vector(embedding_vector)
# print(len(docs))


/Users/sean_liu/miniconda3/envs/langchain/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.' metadata={'source': '../../text_files/state_o